In [3]:
import random
import torch
import csv
import time
import json

import pandas as pd
import numpy as np

from pathlib import Path
from typing import Any, Dict, List
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from Model.sql_encoder import TextSQLEncoder
from Model.Dataloader import prepare_data_for_rf

SEED = 0
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

def write_predictions_and_labels(y_test, predictions, filename="values.scv"):
    """
    Write prediction/label pairs to a CSV-like file.

    Parameters
    ----------
    y_test : array-like
        True labels.
    predictions : array-like
        Predicted values (same length as y_test).
    filename : str
        Output file name (default: 'values.scv').
    """
    if len(y_test) != len(predictions):
        raise ValueError(
            f"Length mismatch: len(y_test)={len(y_test)} "
            f"but len(predictions)={len(predictions)}"
        )

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["prediction", "label"])
        for pred, label in zip(predictions, y_test):
            writer.writerow([float(pred), float(label)])



def run_random_forrest_model(train_df, test_df, database, results_dir):
    test_file_path = Path(f"{results_dir}/results.txt").resolve()
    SEED = 0
    
    torch.manual_seed(SEED)
    random.seed(SEED)
    np.random.seed(SEED)

    # LABEL
    runtime = "Runtime (s)"
    runtime_log = "Runtime_log"
    label = runtime
    
    sql_encoder = TextSQLEncoder()
       
    grid = {
        "n_estimators": [200, 400, 800],
        "max_depth": [None, 10, 20, 40],
    
        "max_features": ["sqrt", "log2"],
    
        "min_samples_split": [2, 5, 10, 20],
        "min_samples_leaf": [1, 2, 4, 20],
    
        "max_leaf_nodes": [None, 1000],  
    
        "bootstrap": [True],
        "random_state": [SEED],
    }
  
    trainer = EncoderTrainerRF(
        train_df=train_df,
        test_df=test_df,
        hyper_param_grid=grid,
        dataframe_label=label,
        results_path=results_dir,
        sql_encoder=sql_encoder,
        schema_encoder=None,
        task_type="Regression",
        seed=SEED
    )
    
    best_params = trainer.optimize_model()
    
    return trainer.evaluate_model()
    

class EncoderTrainerRF:
    """
    Trainer for the Random Forest model using the TriEncoder framework.
    Supports hyperparameter tuning using Grid Search.
    """

    def __init__(self,
                 train_df,
                 test_df,
                 hyper_param_grid,
                 dataframe_label,
                 results_path,
                 sql_encoder,
                 schema_encoder=None,
                 task_type="Regression",
                 seed=SEED):
        
        self.model = None
        self.task_type = task_type
        self.dataframe_label = dataframe_label
        self.train_df = train_df
        self.test_df = test_df
        self.grid = hyper_param_grid

        self.results_path = results_path

        self.sql_encoder = sql_encoder
        self.schema_encoder = schema_encoder

        self.seed = seed
            
        self.model_cls = RandomForestRegressor
        self.scoring_approach = "neg_mean_absolute_error"
        self.train_df = self.create_labels_regression(self.train_df, self.dataframe_label)
        self.test_df = self.create_labels_regression(self.test_df, self.dataframe_label)
        self.grid_search_args = {"estimator":self.model_cls(),"param_grid":self.grid,"cv":5,"n_jobs":-1,"verbose":0, "scoring":self.scoring_approach}
            
        self.X_train, self.y_train, self.train_feature_names, pca_model, query_ids = prepare_data_for_rf(train_df, sql_encoder, schema_encoder)
        self.X_test, self.y_test, self.test_feature_names, _, query_ids = prepare_data_for_rf(test_df, sql_encoder, schema_encoder, pca_model=pca_model)


    def optimize_model(self, params=None):
        """
        Uses GridSearchCV to find the best hyperparameters for the Random Forest model.
        """
        start_time = time.time()

        if params is None:
            grid_search = GridSearchCV(**self.grid_search_args)
            grid_search.fit(self.X_train, self.y_train)
            self.model = grid_search.best_estimator_
            params = grid_search.best_params_
        else:
            self.model = RandomForestRegressor(**params)
            self.model.fit(self.X_train, self.y_train)

        return params

    def train_model(self):
        """
        Trains the Random Forest model using the best hyperparameters found.
        """
        if self.model is None:
            self.model = self.model_cls()
        self.model.fit(self.X_train, self.y_train)

    def evaluate_model(self):
        """
        Evaluates the model on the test set.
        """
        predictions = self.model.predict(self.X_test)

        if self.dataframe_label == "Runtime_log":
            predictions = np.exp(predictions)
            self.y_test = np.exp(self.y_test)
        
        
        mae = mean_absolute_error(self.y_test, predictions)

        def percentile_qerror(y_true, y_pred, percentile, min_runtime=1e-3):
            y_true = np.asarray(y_true, float)
            y_pred = np.asarray(y_pred, float)
        
            mask = (y_true > 0) & (y_pred > 0)
            y_true = y_true[mask]
            y_pred = y_pred[mask]
        
            qerr = np.maximum(y_true, y_pred) / np.maximum(
                np.minimum(y_true, y_pred),
                min_runtime,   
            )

            return float(np.percentile(qerr, percentile))
        med_qerr = percentile_qerror(self.y_test, predictions, 50)  
        p99_qerr = percentile_qerror(self.y_test, predictions, 99)  

        metrics = {f"MAE": mae, f"P50 QError": med_qerr, "P99 QError": p99_qerr}
        
        with open(Path(f"{self.results_path}/results_{self.sql_encoder.type}.txt").resolve(), "w") as f:
            f.write(f"MAE: {mae:.3f}\n")
            f.write(f"P50 QError: {med_qerr:.3f}\n")
            f.write(f"P99 QError: {p99_qerr:.3f}\n")


        return metrics, self.y_test, predictions


    def create_labels_regression(self, df, col_to_use):
        df["label"] = df[col_to_use]
        return df

def load_ndjson_to_dataframe(ndjson_path: str) -> pd.DataFrame:
    df = pd.read_json(ndjson_path, lines=True)
    return df



In [5]:
DATABASES = ["tpcds", "imdb", "ssb"]
RAWFILES = ["Baseline", "Local"]

for DATABASE in DATABASES:

    print(f"================== Database {DATABASE} ====================")
    for RAWFILE in RAWFILES:
        print(f"Type {RAWFILE}")
        experiment_name = f"{RAWFILE}_{DATABASE}"
        experimemt_query_path="../Queries"

        query_path = Path(experimemt_query_path).resolve()
        query_dir = query_path / experiment_name
        query_dir.mkdir(parents=True, exist_ok=True)

        rf_train_path = f"{query_dir}/{DATABASE}_{RAWFILE}_train.ndjson"
        rf_test_path = f"{query_dir}/{DATABASE}_{RAWFILE}_test.ndjson"


        train_df = load_ndjson_to_dataframe(rf_train_path)
        test_df  = load_ndjson_to_dataframe(rf_test_path)

        metrics, labels, preds = run_random_forrest_model(train_df, test_df, database=DATABASE, results_dir=f"./Results/{experiment_name}")

        print(f" mae: {metrics['MAE']} p50: {metrics['P50 QError']} p99: {metrics['P99 QError']}")
        write_predictions_and_labels(labels, preds, filename=f"./Results/{experiment_name}/values.csv")

================== Database tpcds ====================
                   Type Baseline              
 mae: 27.743041607733502 p50: 3.727157953363196 p99: 283.716254014489
                   Type Local              
 mae: 17.40306014869141 p50: 5.643215112190589 p99: 223.60647912807664
================== Database imdb ====================
                   Type Baseline              
 mae: 4.520892242283718 p50: 6.371713195264505 p99: 63.6982845186182
                   Type Local              
 mae: 4.395383971975081 p50: 6.529344561966711 p99: 64.0482651674412
================== Database ssb ====================
                   Type Baseline              
 mae: 43.66075170029343 p50: 2.870081847780856 p99: 429.98917545401184
                   Type Local              
 mae: 11.081604841681557 p50: 2.423744469087278 p99: 115.59088047639584
